# Kronecker Phase Addition

Kronecker phase addition is a term I made up. Note sure if there is any proof of this exact thing in quantum computing literature, but it seems pretty fundamental so it's probably somewhere!

The key idea in Kronecker phase addition is to find the linear operator $F \in \mathbb{R}^{N \times 2^N}$ that maps a layer of phases in a dual rail qubit system to tensor product space. Mathematically representing this tensor product, we have:

$$
\begin{align}
D &\equiv \bigotimes_{n = 0}^{N-1} \begin{bmatrix}
e^{i\phi_0^{(n)}}& 0\\
0 & e^{i\phi_1^{(n)}}
\end{bmatrix}
\end{align},
$$
which we know must also in turn be a diagonal matrix by virtue of the diagonal tensor product rule.

In fact, there is a simple expression for each element of this diagonal that can be proven straightforwardly by induction:

$$
\begin{align}
\angle D_{mm} &= \sum_{n=0}^{N-1} \phi_{0}^{(n)} \cdot \mathbb{1}(m (\mathrm{mod}\ 2^{n + 1}) < 2^{n }) + \phi_{1}^{(n)} \cdot \mathbb{1}(m (\mathrm{mod}\ 2^{n + 1}) \geq 2^{n}),
\end{align}
$$
where $\angle D_{mm}$ represents the phase shift of the $m$th diagonal element of $D$ (where $m \in [1, 2, \ldots 2^N]$) and $\mathbb{1}$ is an indicator function. The operator $F$ satisfies the above expression.

This notebook is meant to run tests to demonstrate the above expression holds for some random selection of $\phi$.

In [1]:
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()

In [2]:
N = 4
phi = np.random.rand(N)

In [4]:
np_tensorprod = np.diag((np.exp(1j * phi[0]), np.exp(1j * (1 - phi[0]))))
for i in range(1, len(phi)):
    np_tensorprod = np.kron(np_tensorprod, np.diag((np.exp(1j * phi[i]), np.exp(1j * (1 - phi[i])))))
np_tensorprod = np.diag(np_tensorprod)

In [6]:
def kronecker_phase_adder(phi_0, phi_1, num_qubits, qubit_idx):
    frequency = 2 ** (num_qubits - qubit_idx - 1)
    block_size = 2 ** qubit_idx
    return tf.reshape(tf.concat((phi_0[qubit_idx] * tf.ones((block_size, frequency), dtype=tf.float64),
                                 phi_1[qubit_idx] * tf.ones((block_size, frequency), dtype=tf.float64)), axis=1),
                      shape=(2 ** num_qubits,))

def phase_shifts_to_tensor_product_space_v2(phi_0, phi_1, num_qubits):
    phi = tf.add_n([kronecker_phase_adder(phi_0, phi_1, num_qubits, qubit_idx)
                    for qubit_idx in range(num_qubits)])
    return tf.complex(tf.cos(phi), tf.sin(phi))

tf_tensorprod = phase_shifts_to_tensor_product_space_v2(phi, 1 - phi, N)

In [7]:
np.allclose(np_tensorprod, tf_tensorprod)

True